In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.special import comb
import ipywidgets as widgets
from ipywidgets import interact

@interact(
    alpha=widgets.FloatSlider(value=2, min=0.1, max=50, step=0.1, description='α (Prior)'),
    beta_param=widgets.FloatSlider(value=2, min=0.1, max=50, step=0.1, description='β (Prior)'),
    successes=widgets.IntSlider(value=5, min=0, max=50, step=1, description='Successes'),
    trials=widgets.IntSlider(value=10, min=1, max=50, step=1, description='Trials')
)
def plot_beta_binomial(alpha, beta_param, successes, trials):
    if successes > trials:
        print("Successes cannot be greater than trials.")
        return

    theta = np.linspace(0.001, 0.999, 500)

    # Prior: Beta(α, β)
    prior = beta.pdf(theta, alpha, beta_param)

    # Likelihood: Binomial PMF for fixed k, variable θ
    likelihood = comb(trials, successes) * (theta**successes) * ((1 - theta)**(trials - successes))
    likelihood = likelihood / np.max(likelihood) * np.max(prior)  # scale to match prior height for comparison

    # Posterior: Beta(α + s, β + f)
    failures = trials - successes
    posterior = beta.pdf(theta, alpha + successes, beta_param + failures)

    # Plot
    fig, axs = plt.subplots(3, 1, figsize=(8, 10))

    axs[0].plot(theta, prior, lw=2, color='blue')
    axs[0].set_title(f"Prior: Beta(α={alpha:.1f}, β={beta_param:.1f})")
    axs[0].set_xlabel(r"$\theta$")
    axs[0].set_ylabel("Density")
    axs[0].grid(True)

    axs[1].plot(theta, likelihood, lw=2, color='orange')
    axs[1].set_title(f"Likelihood (Binomial PMF): k={successes}, n={trials}")
    axs[1].set_xlabel(r"$\theta$")
    axs[1].set_ylabel("Likelihood (scaled)")
    axs[1].grid(True)

    axs[2].plot(theta, posterior, lw=2, color='green')
    axs[2].set_title(f"Posterior: Beta(α={alpha + successes:.1f}, β={beta_param + failures:.1f})")
    axs[2].set_xlabel(r"$\theta$")
    axs[2].set_ylabel("Density")
    axs[2].grid(True)

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=2.0, description='α (Prior)', max=50.0, min=0.1), FloatSlider(value=2.…

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma, poisson
import ipywidgets as widgets
from ipywidgets import interact

@interact(
    alpha=widgets.FloatSlider(value=2, min=0.1, max=50, step=0.1, description='α (Prior)'),
    beta_param=widgets.FloatSlider(value=1, min=0.1, max=20, step=0.1, description='β (Prior)'),
    total_counts=widgets.IntSlider(value=12, min=0, max=100, step=1, description='Total x'),
    n_obs=widgets.IntSlider(value=5, min=1, max=50, step=1, description='n obs')
)
def plot_gamma_poisson(alpha, beta_param, total_counts, n_obs):
    theta = np.linspace(0.001, 20, 500)

    # Prior: Gamma(α, β)
    prior = gamma.pdf(theta, a=alpha, scale=1/beta_param)

    # Likelihood: Product of Poisson(x_i | θ) ∝ θ^sum(x) * e^(-nθ) (ignoring constants)
    likelihood = (theta**total_counts) * np.exp(-n_obs * theta)
    likelihood /= np.max(likelihood)  # normalize for plotting
    likelihood *= np.max(prior)  # scale to prior height for comparison

    # Posterior: Gamma(α + sum x_i, β + n)
    posterior = gamma.pdf(theta, a=alpha + total_counts, scale=1/(beta_param + n_obs))

    # Plot
    fig, axs = plt.subplots(3, 1, figsize=(8, 10))

    axs[0].plot(theta, prior, lw=2, color='blue')
    axs[0].set_title(f"Prior: Gamma(α={alpha:.1f}, β={beta_param:.1f})")
    axs[0].set_xlabel(r"$\theta$")
    axs[0].set_ylabel("Density")
    axs[0].grid(True)

    axs[1].plot(theta, likelihood, lw=2, color='orange')
    axs[1].set_title(f"Likelihood (Poisson, ∑x={total_counts}, n={n_obs})")
    axs[1].set_xlabel(r"$\theta$")
    axs[1].set_ylabel("Scaled Likelihood")
    axs[1].grid(True)

    axs[2].plot(theta, posterior, lw=2, color='green')
    axs[2].set_title(f"Posterior: Gamma(α={alpha + total_counts:.1f}, β={beta_param + n_obs:.1f})")
    axs[2].set_xlabel(r"$\theta$")
    axs[2].set_ylabel("Density")
    axs[2].grid(True)

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=2.0, description='α (Prior)', max=50.0, min=0.1), FloatSlider(value=1.…

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from ipywidgets import interact

@interact(
    mu0=widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='μ₀ (Prior Mean)'),
    tau=widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='τ (Prior Std)'),
    sigma=widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='σ (Data Std)'),
    n=widgets.IntSlider(value=10, min=1, max=100, step=1, description='n Obs'),
    xbar=widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='x̄ (Mean of Data)')
)
def plot_normal_normal(mu0, tau, sigma, n, xbar):
    theta = np.linspace(-10, 10, 500)

    # Prior
    prior = norm.pdf(theta, loc=mu0, scale=tau)

    # Likelihood (viewed as a function of μ)
    likelihood_var = sigma**2 / n
    likelihood = norm.pdf(theta, loc=xbar, scale=np.sqrt(likelihood_var))
    likelihood = likelihood / np.max(likelihood) * np.max(prior)

    # Posterior parameters
    tau2 = tau**2
    sigma2 = sigma**2
    post_var = 1 / (1 / tau2 + n / sigma2)
    post_mean = post_var * (mu0 / tau2 + n * xbar / sigma2)
    posterior = norm.pdf(theta, loc=post_mean, scale=np.sqrt(post_var))

    # Plot
    fig, axs = plt.subplots(3, 1, figsize=(8, 10))

    axs[0].plot(theta, prior, lw=2, color='blue')
    axs[0].set_title(f"Prior: Normal(μ₀={mu0:.2f}, τ={tau:.2f})")
    axs[0].set_xlabel(r"$\theta$")
    axs[0].set_ylabel("Density")
    axs[0].grid(True)

    axs[1].plot(theta, likelihood, lw=2, color='orange')
    axs[1].set_title(f"Likelihood: Normal(x̄={xbar:.2f}, σ²/n={likelihood_var:.2f})")
    axs[1].set_xlabel(r"$\theta$")
    axs[1].set_ylabel("Scaled Likelihood")
    axs[1].grid(True)

    axs[2].plot(theta, posterior, lw=2, color='green')
    axs[2].set_title(f"Posterior: Normal(μ={post_mean:.2f}, σ={np.sqrt(post_var):.2f})")
    axs[2].set_xlabel(r"$\theta$")
    axs[2].set_ylabel("Density")
    axs[2].grid(True)

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=0.0, description='μ₀ (Prior Mean)', max=10.0, min=-10.0), FloatSlider(…

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import invgamma, norm
import ipywidgets as widgets
from ipywidgets import interact

@interact(
    mu0=widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='μ₀'),
    kappa0=widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='κ₀'),
    alpha=widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='α'),
    beta_param=widgets.FloatSlider(value=2, min=0.1, max=20, step=0.1, description='β'),
    xbar=widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='x̄'),
    s2=widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='s²'),
    n=widgets.IntSlider(value=10, min=1, max=100, step=1, description='n')
)
def plot_normal_unknown_variance(mu0, kappa0, alpha, beta_param, xbar, s2, n):
    # Grids
    mu_grid = np.linspace(-10, 10, 500)
    sigma2_grid = np.linspace(0.1, 20, 500)

    # Prior σ² ~ Inverse-Gamma
    prior_sigma2 = invgamma.pdf(sigma2_grid, a=alpha, scale=beta_param)

    # Posterior parameters
    alpha_post = alpha + n / 2
    beta_post = beta_param + 0.5 * (n - 1) * s2 + (kappa0 * n * (xbar - mu0)**2) / (2 * (kappa0 + n))
    kappa_post = kappa0 + n
    mu_post = (kappa0 * mu0 + n * xbar) / kappa_post

    # Posterior σ² ~ Inverse-Gamma
    posterior_sigma2 = invgamma.pdf(sigma2_grid, a=alpha_post, scale=beta_post)

    # Expectation of σ² posterior
    sigma2_mean_post = beta_post / (alpha_post - 1) if alpha_post > 1 else beta_post / 1e-3
    posterior_mu = norm.pdf(mu_grid, loc=mu_post, scale=np.sqrt(sigma2_mean_post / kappa_post))

    # Prior μ ~ Normal(μ₀, σ² / κ₀), σ² = E[prior]
    sigma2_mean_prior = beta_param / (alpha - 1) if alpha > 1 else beta_param / 1e-3
    prior_mu = norm.pdf(mu_grid, loc=mu0, scale=np.sqrt(sigma2_mean_prior / kappa0))

    # Likelihood over μ: x̄ ~ Normal(μ, σ²/n), con s² como proxy de σ²
    likelihood_mu = norm.pdf(mu_grid, loc=xbar, scale=np.sqrt(s2 / n))
    likelihood_mu = likelihood_mu / np.max(likelihood_mu) * np.max(prior_mu)  # scaled for visual match

    # Plotting
    fig, axs = plt.subplots(4, 1, figsize=(8, 14))

    axs[0].plot(mu_grid, prior_mu, lw=2, color='blue')
    axs[0].set_title(f"Prior on μ ~ Normal(μ₀={mu0:.2f}, σ²={sigma2_mean_prior:.2f}/κ₀)")
    axs[0].set_xlabel(r"$\mu$")
    axs[0].set_ylabel("Density")
    axs[0].grid(True)

    axs[1].plot(mu_grid, likelihood_mu, lw=2, color='orange')
    axs[1].set_title(f"Likelihood ~ Normal(μ; x̄={xbar:.2f}, s²/n={s2/n:.2f})")
    axs[1].set_xlabel(r"$\mu$")
    axs[1].set_ylabel("Scaled Likelihood")
    axs[1].grid(True)

    axs[2].plot(mu_grid, posterior_mu, lw=2, color='green')
    axs[2].set_title(f"Posterior on μ ~ Normal(μ={mu_post:.2f}, σ²={sigma2_mean_post:.2f}/κₙ)")
    axs[2].set_xlabel(r"$\mu$")
    axs[2].set_ylabel("Density")
    axs[2].grid(True)

    axs[3].plot(sigma2_grid, prior_sigma2, lw=2, color='blue', label="Prior σ²")
    axs[3].plot(sigma2_grid, posterior_sigma2, lw=2, color='green', label="Posterior σ²")
    axs[3].set_title("σ² ~ Inverse-Gamma")
    axs[3].set_xlabel(r"$\sigma^2$")
    axs[3].set_ylabel("Density")
    axs[3].legend()
    axs[3].grid(True)

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=0.0, description='μ₀', max=10.0, min=-10.0), FloatSlider(value=1.0, de…